In [1]:
!rm -rf Team3_repo

In [2]:
!git clone https://github.com/Jungminchae/Team3_repo.git

Cloning into 'Team3_repo'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 442 (delta 0), reused 1 (delta 0), pack-reused 437
Receiving objects: 100% (442/442), 9.35 MiB | 34.46 MiB/s, done.
Resolving deltas: 100% (222/222), done.


In [3]:
# !pip install wandb

In [4]:
# import wandb

# wandb.init(project="EfficientFood", reinit=True)
# wandb.log({"loss" : loss, "val_loss":val_loss, "val_acc" : val_accuracy})

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import sys
sys.path.append('/content/Team3_repo/hackathon/notebooks')
sys.path.append('/content/Team3_repo/hackathon/food_30')

In [7]:
import os
from glob import glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from dataAugmentation import DataPreprocessing
from utils import CosineAnnealingScheduler
tf.random.set_seed(2020)

# 1. tfrecord load with aug

In [8]:
DATASET_SIZE = 92478
BATCH_SIZE = 128
IMAGE_SIZE = 224

MODEL_NAME = "Model_01"
## 모델 체크포인트 경로 설정
mc_dir_path = '/content/drive/Shareddrives/GOFOODA/data/models_with100/EfficientNetB0_alb_checkpoint/'
if not os.path.exists(mc_dir_path):
    print('making directory')
    os.makedirs(mc_dir_path)

In [9]:
tfr_train_dir = '/content/drive/Shared drives/GOFOODA/data/food_data.tfr'
dp_new = DataPreprocessing(tfr_train_dir,image_size=IMAGE_SIZE, batch_size=BATCH_SIZE, buffer_size=DATASET_SIZE, dataset_size=DATASET_SIZE, num_class=100)
train_ds, valid_ds = dp_new()

True in data_alb func


# 모델 생성
---
기존의 EfficientNetB0모델을 가져온다.

In [10]:
train_size = int(0.8*DATASET_SIZE)
valid_size = int(0.2*DATASET_SIZE)

In [11]:
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
    )
base_model.trainable = False

base_model.layers[-5:]

In [12]:
GAP_layer = tf.keras.layers.GlobalAveragePooling2D()
BN_layer = tf.keras.layers.BatchNormalization()
#dense_layer = tf.keras.layers.Dense(512, activation='relu')
prediction_layer = tf.keras.layers.Dense(100, activation='softmax')

model = tf.keras.Sequential([
                             base_model,
                             GAP_layer,
                             BN_layer,
                             prediction_layer
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 7, 7, 1280)        4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1280)              5120      
_________________________________________________________________
dense (Dense)                (None, 100)               128100    
Total params: 4,182,791
Trainable params: 130,660
Non-trainable params: 4,052,131
_________________________________________________________________


In [11]:
# EarlyStopping
import datetime

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

ca = CosineAnnealingScheduler(T_max=100, eta_max=1e-3, eta_min=3e-5, verbose=1)
cd = tf.keras.experimental.CosineDecay(1e-3, decay_steps=train_size, alpha=0.0)
log_dir = "/content/drive/Shareddrives/GOFOODA/data/models_with100/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [14]:
#weight warm up
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2),
    loss = tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

In [15]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(
    train_ds,
    epochs=20,
    validation_data = valid_ds,
    steps_per_epoch=train_size//BATCH_SIZE,
    callbacks=[es]
)

warmup_model_path = os.path.join(mc_dir_path, 'WARPUP_'+MODEL_NAME)
model.save(warmup_model_path)

Epoch 1/20
577/577 [==============================] - 272s 472ms/step - loss: 1.6798 - accuracy: 0.5791 - val_loss: 0.9560 - val_accuracy: 0.7226
Epoch 2/20
577/577 [==============================] - 269s 467ms/step - loss: 1.1685 - accuracy: 0.6664 - val_loss: 0.8833 - val_accuracy: 0.7391
Epoch 3/20
577/577 [==============================] - 268s 465ms/step - loss: 1.1125 - accuracy: 0.6819 - val_loss: 0.8425 - val_accuracy: 0.7543
Epoch 4/20
577/577 [==============================] - 268s 465ms/step - loss: 1.0860 - accuracy: 0.6895 - val_loss: 0.8349 - val_accuracy: 0.7525
Epoch 5/20
577/577 [==============================] - 267s 463ms/step - loss: 1.0681 - accuracy: 0.6941 - val_loss: 0.7934 - val_accuracy: 0.7688
Epoch 6/20
577/577 [==============================] - 272s 471ms/step - loss: 1.0679 - accuracy: 0.6954 - val_loss: 0.7753 - val_accuracy: 0.7693
Epoch 7/20
577/577 [==============================] - 275s 477ms/step - loss: 1.0536 - accuracy: 0.6998 - val_loss: 0.7538 -

### Done top layer weight warm-up Let's do fine tune


In [14]:
mc_dir_path = '/content/drive/Shareddrives/GOFOODA/data/models_with100/EfficientNetB0_alb_checkpoint/'
if not os.path.exists(mc_dir_path):
    print('making directory')
    os.makedirs(mc_dir_path)
# mc_path = os.path.join(mc_dir_path, 'EfficientNet_alb.h5')
# print(f"mc_path : {mc_path}")

# ModelCheckpoint
mc = tf.keras.callbacks.ModelCheckpoint(
    filepath=mc_dir_path+'final_alb_tuned_{epoch}-{val_loss:.2f}-{val_accuracy:.2f}.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1,
    )

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [12]:
# load checkpoint
model = tf.keras.models.load_model('/content/drive/Shareddrives/GOFOODA/data/models_with100/EfficientNetB0_alb_checkpoint/alb_tuned_15-0.15-0.95.h5')

In [13]:
#after weight warm up
#now unfreeze some layers from the top
def unfreeze_model(model):
  for layer in model.layers[-8:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
      layer.trainable = True
  return model

model = unfreeze_model(model)

#complie model
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    epochs=100,
    validation_data = valid_ds,
    steps_per_epoch=train_size//BATCH_SIZE,
    callbacks=[es, mc, ca]
)


Epoch 00001: CosineAnnealingScheduler setting learning rate to 0.001.
Epoch 1/100
121/577 [=====>........................] - ETA: 7:57 - loss: 0.1711 - accuracy: 0.9449

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')

acc_ax.plot(history.history['accuracy'], 'b', label='train acc')
acc_ax.plot(history.history['val_accuracy'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')
plt.title(max(history.history['val_accuracy']))
plt.savefig(mc_dir_path + 'result.png')
plt.show()

In [ ]:
model.save(os.path.join(mc_dir_path, 'EfficientNetB0_aug.h5'))